In [128]:
import nltk
tokens = nltk.word_tokenize("DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY WERE OKAY WITH IT. JUST NOT WHAT I EXPECTED")
print(nltk.pos_tag(tokens))

pos = pd.DataFrame(pos_tag(tokens))
pos = " ".join(list(pos[pos[1].str.contains("JJ")].iloc[:,0]))
pos

[('DO', 'VBP'), ("N'T", 'NNP'), ('CARE', 'NNP'), ('FOR', 'IN'), ('IT', 'NNP'), ('.', '.'), ('GAVE', 'VB'), ('IT', 'NNP'), ('AS', 'IN'), ('A', 'NNP'), ('GIFT', 'NNP'), ('AND', 'NNP'), ('THEY', 'NNP'), ('WERE', 'NNP'), ('OKAY', 'NNP'), ('WITH', 'NNP'), ('IT', 'NNP'), ('.', '.'), ('JUST', 'NNP'), ('NOT', 'NNP'), ('WHAT', 'NNP'), ('I', 'PRP'), ('EXPECTED', 'VBP')]


''

In [8]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [72]:
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk import pos_tag
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import re
from contractions import contractions_dict
from string import punctuation
pd.set_option('display.max_colwidth', -1)

In [10]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Cell_Phones_and_Accessories_5.json.gz')

df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [16]:
df = df.dropna(subset = ['reviewText','summary'])
df.isna().sum()

overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style              522684
reviewerName          133
reviewText              0
summary                 0
unixReviewTime          0
vote              1035232
image             1100212
dtype: int64

In [17]:
df['overall'].value_counts()

5.0    706102
4.0    184293
3.0     98189
1.0     81469
2.0     57153
Name: overall, dtype: int64

In [190]:
df_negative_reviews = df[df['overall']<3].iloc[:10000]
df_positive_reviews = df[df['overall']>3].iloc[:10000]

In [36]:
df_new = df.iloc[:100]

In [62]:
def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text



def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)
# text = "Hello! how are you doing?"
# print (strip_punctuation(text))

# def main():
#     text = """I ain't going there. You'll have to go alone."""
    
#     text=expand_contractions(text,contractions_dict)
# #     sentences = nltk.sent_tokenize(text)
# #     tokenized_sentences = nltk.word_tokenize(text)
    
#     print (text)


# if __name__ == '__main__':
#     main()

Hello how are you doing


In [188]:
from nltk.tokenize import RegexpTokenizer
def cleanme(txt):
    sent = txt.lower()
    sent_expanded_contractions = expand_contractions(sent,contractions_dict)
    sent_expanded_contractions = re.sub(r'(?<=[.,])(?=[^\s])', r' ', sent_expanded_contractions)
    sent_without_punct = strip_punctuation(sent_expanded_contractions)
    TOKENIZER = RegexpTokenizer('(?u)\W+|\$[\d\.]+|\S+')
#     wrds = TOKENIZER.tokenize(sent_without_punct)
    wrds = word_tokenize(sent_without_punct)
    
    clwrds = [w for w in wrds if not w in stopwords.words('english')]
    ln = len(clwrds)
    pos = pd.DataFrame(pos_tag(wrds))
    pos = (" ".join(list(pos[pos[1].str.contains("JJ")].iloc[:,0]))).split(" ")
    l2 = ["i","you","me"]
    pos = [x for x in pos if x not in l2]
    pos
    rt = [ln, " ".join(clwrds), " ".join(pos)]
    return(rt)

In [157]:
tmp = list()
for i in range(5):
    tmp.append(cleanme(df.iloc[i,:]['reviewText']))
tmp = pd.DataFrame(tmp)
tmp.columns = ['reviewlen', 'cleanrev', 'adjreview']

print(tmp.head())

   reviewlen  \
0  22          
1  10          
2  23          
3  5           
4  12          

                                                                                                                                                       cleanrev  \
0  looks even better person careful drop phone often rhinestones fall duh decorative case protective say fits perfectly securely phone overall pleased purchase   
1  want spend whole lot cash want great deal shop buy                                                                                                             
2  case came time love design actually missing 2 studs nothing noticeable studding almost bit sloppy around bow noticeable put phone yet notice far               
3  care gave gift okay expected                                                                                                                                   
4  liked cute studs fall easily protect phone would recommended buy like looks          

In [191]:
tmp = list()
for i in range(1000):
    tmp.append(cleanme(df_negative_reviews.iloc[i,:]['reviewText']))
tmp = pd.DataFrame(tmp)
tmp.columns = ['reviewlen', 'cleanrev', 'adjreview']

(tmp.head())


df_negative_reviews_new = df_negative_reviews.reset_index()
# df_negative_reviews_new.drop(['reviewlen', 'cleanrev', 'adjreview'], axis=1, inplace=True)
df_negative_reviews_new = pd.concat([df_negative_reviews_new,tmp], axis=1)
df_negative_reviews_new = df_negative_reviews_new[['overall','reviewText','summary','reviewlen', 'cleanrev', 'adjreview']]
df_negative_reviews_new

,overall,reviewText,summary,reviewlen,cleanrev,adjreview
0,2.0,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY WERE OKAY WITH IT. JUST NOT WHAT I EXPECTED.,CASE,5.0,care gave gift okay expected,
1,2.0,The product looked exactly like the picture and it was very nice. However only days later it fell apart. I'm very disappointed with the quality of the product.,Not so happy,14.0,product looked exactly like picture nice however days later fell apart disappointed quality product,nice disappointed
2,1.0,DO NOT BUY! this item is seriously cheap as heck. not worth buying it at all. I didn't even get to use it and it was already losing all of its gems. I wish I got my money back on this item!!,WORST ITEM!,18.0,buy item seriously cheap heck worth buying even get use already losing gems wish got money back item,cheap worth wish
3,1.0,Very cheap broke the first time we put it on :( It was pretty but very cheaply made!! Too bad as it is cute,cheap plastic,10.0,cheap broke first time put pretty cheaply made bad cute,cheap first bad cute
4,2.0,"I used this case for not even a week and the bow came off. I loved it so pretty, but wish it would of stayed together.",....,12.0,used case even week bow came loved pretty wish would stayed together,wish
5,1.0,The bow fell off multiple times I had to keep gluing it back on until it wouldn't let me glue anymore. of course studs fell off but the bow it what makes it.,Don't waste your money or invest in glue,16.0,bow fell multiple times keep gluing back would let glue anymore course studs fell bow makes,multiple
6,1.0,"I was very excited when I first got this case. I loved the color and the feel of it. Then I started noticing stuff on my hands and realized the ""rubber"" peels right off the phone just from normal use. As the saying goes you get what you pay for.",A waste of money!,22.0,excited first got case loved color feel started noticing stuff hands realized rubber peels right phone normal use saying goes get pay,excited normal
7,1.0,Sent case that didn't fit,One Star,3.0,sent case fit,
8,2.0,The picture looks waaayyy better than the actual item however its strong and serves its purpose.\nThe finish is matte not glossy as the picture shows and has a cheap look to it.,"Not so great advertisement, but ok, i guess.",17.0,picture looks waaayyy better actual item however strong serves purpose finish matte glossy picture shows cheap look,better actual strong matte glossy cheap
9,2.0,"I loved the case when I first received it but shortly after, the case started to peel off at first i did not know what it was until i looked on the back of the case and it was missing spots. Guess sometimes a good deal is not really a good deal.would not purchase.",Loved it at first,24.0,loved case first received shortly case started peel first know looked back case missing spots guess sometimes good deal really good deal would purchase,first good good


In [209]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')

txt = df_negative_reviews_new.reviewText.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(txt)
lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in words]
# stemmed_word = [snowball_stemmer.stem(word) for word in words]
bgs = nltk.bigrams(lemmatized_word)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
fdist.most_common(20)

# word_dist = nltk.FreqDist(stemmed_word)
# print(word_dist)
# word_dist.most_common(20)

[(('.', 'i'), 9316),
 (('.', 'the'), 4612),
 (('.', 'it'), 3296),
 (('the', 'phone'), 3237),
 ((',', 'but'), 2812),
 (('of', 'the'), 2764),
 ((',', 'and'), 2452),
 ((',', 'i'), 2396),
 (('on', 'the'), 2245),
 (('in', 'the'), 1958),
 (('do', "n't"), 1956),
 (('i', 'have'), 1922),
 (('.', 'this'), 1855),
 (('it', "'s"), 1844),
 (('it', '.'), 1802),
 ((',', 'the'), 1788),
 (('it', 'wa'), 1750),
 (('it', 'is'), 1747),
 ((',', 'it'), 1737),
 (('!', '!'), 1728)]

In [186]:
tmp = list()
for i in range(1000):
    tmp.append(cleanme(df_positive_reviews.iloc[i,:]['reviewText']))
tmp = pd.DataFrame(tmp)
tmp.columns = ['reviewlen', 'cleanrev', 'adjreview']

(tmp.head())


df_positive_reviews_new = df_positive_reviews.reset_index()
# df_negative_reviews_new.drop(['reviewlen', 'cleanrev', 'adjreview'], axis=1, inplace=True)
df_positive_reviews_new = pd.concat([df_positive_reviews_new,tmp], axis=1)
df_positive_reviews_new = df_positive_reviews_new[['overall','reviewText','summary','reviewlen', 'cleanrev', 'adjreview']]
df_positive_reviews_new.head()

,overall,reviewText,summary,reviewlen,cleanrev,adjreview
0,5.0,"Looks even better in person. Be careful to not drop your phone so often because the rhinestones will fall off (duh). More of a decorative case than it is protective, but I will say that it fits perfectly and securely on my phone. Overall, very pleased with this purchase.",Can't stop won't stop looking at it,22.0,looks even better person careful drop phone often rhinestones fall duh decorative case protective say fits perfectly securely phone overall pleased purchase,more
1,5.0,When you don't want to spend a whole lot of cash but want a great deal...this is the shop to buy from!,1,10.0,want spend whole lot cash want great deal shop buy,
2,4.0,"I liked it because it was cute, but the studs fall off easily and to protect a phone this would not be recommended. Buy if you just like it for looks.",Cute!,12.0,liked cute studs fall easily protect phone would recommended buy like looks,
3,5.0,"It is a very cute case. None of the jewels have fallen off and the bow is glued on very well. Came in the time frame that was given and I'm happy with this purchase. The only thing you might want to know is, I carry my phone in my jeans and it does fit very tightly because it is a big bow. You have to be careful but worth it.",Cute case,27.0,cute case none jewels fallen bow glued well came time frame given happy purchase thing might want know carry phone jeans fit tightly big bow careful worth,
4,4.0,"I really love this case... you have to keep your phone face down all the time, but it's pretty good quality. Some stones come off from normal use, unless you keep it in your purse all day.",Pretty Cute!,18.0,really love case keep phone face time pretty good quality stones come normal use unless keep purse day,


In [210]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')

txt = df_positive_reviews_new.cleanrev.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(txt)
lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in words]
# stemmed_word = [snowball_stemmer.stem(word) for word in words]

bgs = nltk.bigrams(lemmatized_word)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
fdist.most_common(30)
# for k,v in fdist.items():
#     print (k,v)
    
# word_dist = nltk.FreqDist(lemmatized_word)
# print(word_dist)
# word_dist.most_common(20)

[(('work', 'great'), 60),
 (('battery', 'life'), 60),
 (('work', 'well'), 47),
 (('sound', 'quality'), 47),
 (('cell', 'phone'), 39),
 (('great', 'product'), 33),
 (('bluetooth', 'headset'), 31),
 (('usb', 'port'), 27),
 (('usb', 'hub'), 25),
 (('would', 'recommend'), 23),
 (('highly', 'recommend'), 22),
 (('easy', 'use'), 22),
 (('typehidden', 'name'), 20),
 (('name', 'valuehttpsimagesna'), 20),
 (('valuehttpsimagesna', 'sslimagesamazon'), 20),
 (('use', 'phone'), 18),
 (('ear', 'bud'), 18),
 (('look', 'like'), 17),
 (('work', 'perfectly'), 17),
 (('ear', 'piece'), 17),
 (('around', 'neck'), 17),
 (('new', 'phone'), 16),
 (('battery', 'last'), 16),
 (('one', 'best'), 16),
 (('make', 'sure'), 16),
 (('long', 'time'), 15),
 (('phone', 'case'), 15),
 (('even', 'though'), 15),
 (('charge', 'phone'), 15),
 (('much', 'better'), 15)]

In [158]:
df_new = df_new.reset_index()
df_new.drop(['reviewlen', 'cleanrev', 'adjreview'], axis=1, inplace=True)
df_new = pd.concat([df_new,tmp], axis=1)
df_new = df_new[['overall','reviewText','summary','reviewlen', 'cleanrev', 'adjreview']]
df_new.head()

,overall,reviewText,summary,reviewlen,cleanrev,adjreview
0,5.0,"Looks even better in person. Be careful to not drop your phone so often because the rhinestones will fall off (duh). More of a decorative case than it is protective, but I will say that it fits perfectly and securely on my phone. Overall, very pleased with this purchase.",Can't stop won't stop looking at it,22.0,looks even better person careful drop phone often rhinestones fall duh decorative case protective say fits perfectly securely phone overall pleased purchase,careful more decorative protective overall pleased
1,5.0,When you don't want to spend a whole lot of cash but want a great deal...this is the shop to buy from!,1,10.0,want spend whole lot cash want great deal shop buy,whole great
2,3.0,"so the case came on time, i love the design. I'm actually missing 2 studs but nothing too noticeable the studding is almost a bit sloppy around the bow, but once again not too noticeable. I haven't put in my phone yet so this is just what I've notice so far",Its okay,23.0,case came time love design actually missing 2 studs nothing noticeable studding almost bit sloppy around bow noticeable put phone yet notice far,noticeable sloppy noticeable
3,2.0,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY WERE OKAY WITH IT. JUST NOT WHAT I EXPECTED.,CASE,5.0,care gave gift okay expected,
4,4.0,"I liked it because it was cute, but the studs fall off easily and to protect a phone this would not be recommended. Buy if you just like it for looks.",Cute!,12.0,liked cute studs fall easily protect phone would recommended buy like looks,cute studs


In [179]:
txt = df_positive_reviews_new.adjreview.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(words)
print(word_dist)
word_dist.most_common(20)

<FreqDist with 633 samples and 2477 outcomes>


[('great', 159),
 ('good', 117),
 ('other', 56),
 ('more', 47),
 ('nice', 46),
 ('little', 44),
 ('new', 41),
 ('last', 37),
 ('happy', 30),
 ('much', 29),
 ('easy', 29),
 ('long', 28),
 ('full', 27),
 ('many', 25),
 ('few', 25),
 ('best', 25),
 ('original', 23),
 ('only', 22),
 ('old', 22),
 ('same', 21)]

In [202]:
raw = "I am Ankit Taparia"

tokens = nltk.word_tokenize(raw)


bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
    print (k,v)

('I', 'am') 1
('am', 'Ankit') 1
('Ankit', 'Taparia') 1
